Now we parse the content of the JSON file with the QASM circuits

In [10]:
using JSON
using ITensors



In [2]:

# Function to parse and organize the JSON data
function parse_subcircuits(json_filename::String)

    json_data = JSON.parsefile(json_filename)
    
    subexp_dict = Dict{String, Vector{Dict{String, Any}}}()
    
    for entry in json_data
        category = entry["Subexperiment"]
        index = entry["Subcircuit"]
        operations = entry["Operations"]
        operations_raw = entry["Operations"]
        
        processed_operations = Vector{Dict{String, Any}}()
        for op in operations_raw
            name = op["Name"]
            qubits = op["Qubits"]
            angle_array = op["Angle"]
            
            angle = if length(angle_array) == 1
                        angle_array[1]
                    else
                        NaN  
                    end
            
            processed_op = Dict(
                "Name" => name,
                "Angle" => angle,
                "Qubits" => qubits
            )
            
            push!(processed_operations, processed_op)
        end
        
        # Create a subcircuit dictionary
        subcircuit = Dict(
            "Subcircuit" => index,
            "Operations" => processed_operations
        )
        
        if haskey(subexp_dict, category)
            push!(subexp_dict[category], subcircuit)
        else
            subexp_dict[category] = [subcircuit]
        end
    end
    
    sorted_categories = sort(collect(keys(subexp_dict)))
    
    organized_data = Dict{String, Vector{Dict{String, Any}}}()
    
    for category in sorted_categories
        sorted_subcircuits = sort(subexp_dict[category], by = sc -> sc["Subcircuit"])
        organized_data[category] = sorted_subcircuits
    end
    
    return organized_data
end

function display_organized_data(organized_data::Dict{String, Vector{Dict{String, Any}}})
    for (category, subcircuits) in organized_data
        println("Subexperiment Category: ", category)
        for subcircuit in subcircuits
            println("  Subcircuit Index: ", subcircuit["Subcircuit"])
            for op in subcircuit["Operations"]
                println("    Operation: ", op["Name"])
                println("      Angle: ", op["Angle"])
                println("      Qubits: ", op["Qubits"])
            end
        end
    end
end

json_filename = "subcircuits.json"  # Ensure this file is in the current directory or provide the correct path
organized_data = parse_subcircuits(json_filename)


Dict{String, Vector{Dict{String, Any}}} with 2 entries:
  "B" => [Dict("Subcircuit"=>0, "Operations"=>Dict{String, Any}[Dict("Angle"=>0…
  "A" => [Dict("Subcircuit"=>0, "Operations"=>Dict{String, Any}[Dict("Angle"=>0…

In [11]:
#Read and parse the json
file_content = read("circuits.json", String)
qasm_list = JSON.parse(file_content)

# Now, qasm_list is an array of QASM strings
print(qasm_list)

Any["OPENQASM 3.0;\ninclude \"stdgates.inc\";\ngate rxx(p0) _gate_q_0, _gate_q_1 {\n  h _gate_q_0;\n  h _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  rz(p0) _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  h _gate_q_1;\n  h _gate_q_0;\n}\nbit[5] observable_measurements;\nbit[1] qpd_measurements;\nqubit[5] q2;\nrxx(0.4) q2[0], q2[1];\nrxx(0.4) q2[1], q2[2];\nrz(0.4) q2[1];\nrxx(0.4) q2[2], q2[3];\nrz(0.4) q2[2];\nrxx(0.4) q2[3], q2[4];\nrxx(0.4) q2[0], q2[4];\nrz(0.4) q2[0];\nrz(0.4) q2[3];\nrz(0.4) q2[4];\nobservable_measurements[0] = measure q2[0];\nobservable_measurements[1] = measure q2[1];\nobservable_measurements[2] = measure q2[2];\nobservable_measurements[3] = measure q2[3];\nobservable_measurements[4] = measure q2[4];\n", "OPENQASM 3.0;\ninclude \"stdgates.inc\";\ngate rxx(p0) _gate_q_0, _gate_q_1 {\n  h _gate_q_0;\n  h _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  rz(p0) _gate_q_1;\n  cx _gate_q_0, _gate_q_1;\n  h _gate_q_1;\n  h _gate_q_0;\n}\nbit[5] observable_measurements;\nbit[1] qpd_mea

In [12]:
function parse_qasm(qasm_string::String)
    lines = split(qasm_string, "\n")
    n_qubits = 0
    gates = []

    for line in lines
        line = strip(split(line, "//")[1])
        if isempty(line)
            continue
        end

        # Qubit number
        if startswith(line, "qubit[")
            m = match(r"qubit\[(\d+)\]\s+\w+;", line)
            if m !== nothing
                n_qubits = parse(Int, m.captures[1])
            end

        # RXX Gate
        elseif startswith(line, "rxx(")
            m = match(r"rxx\((.*?)\)\s+q10\[(\d+)\],\s*q10\[(\d+)\];", line)
            if m !== nothing
                θ = parse(Float64, m.captures[1])
                q1 = parse(Int, m.captures[2]) + 1
                q2 = parse(Int, m.captures[3]) + 1
                push!(gates, (:rxx, θ, q1, q2))
            end

        # RZ Gate
        elseif startswith(line, "rz(")
            m = match(r"rz\((.*?)\)\s+q10\[(\d+)\];", line)
            if m !== nothing
                θ = parse(Float64, m.captures[1])
                q = parse(Int, m.captures[2]) + 1
                push!(gates, (:rz, θ, q))
            end

        # H Gate
        elseif startswith(line, "h ")
            m = match(r"h\s+q10\[(\d+)\];", line)
            if m !== nothing
                q = parse(Int, m.captures[1]) + 1
                push!(gates, (:h, q))
            end

        # CX Gate
        elseif startswith(line, "cx ")
            m = match(r"cx\s+q10\[(\d+)\],\s*q10\[(\d+)\];", line)
            if m !== nothing
                q1 = parse(Int, m.captures[1]) + 1
                q2 = parse(Int, m.captures[2]) + 1
                push!(gates, (:cx, q1, q2))
            end

        # QPD Measurement
        elseif startswith(line, "qpd_measurements[")
            m = match(r"qpd_measurements\[(\d+)\]\s*=\s*measure\s+q10\[(\d+)\];", line)
            if m !== nothing
                # meas_index = parse(Int, m.captures[1]) + 1
                qubit = parse(Int, m.captures[2]) + 1
                # push!(gates, (:measure_qpd, meas_index, qubit))
                push!(gates, (:measure_qpd, qubit))
            end

        end
    end

    return n_qubits, gates
end


parse_qasm (generic function with 1 method)

In [14]:
qasm_str = qasm_list[1]
n_qubits, gates = parse_qasm(qasm_str);
print(n_qubits)

5

In [ ]:
function create_sites(n::Int)
    return [Index(2, "Qubit,i$i") for i in 1:n]
end

create_sites (generic function with 1 method)

In [ ]:
# Rz Gate with angle θ
function rz_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rz_mpo = MPO(sites)
    for j in 1:N
        rz_mpo[j] = op("Id", sites[j])
    end
    rz_mpo[i] = op("Rz", sites[i]; θ=θ)
    return rz_mpo
end

# H 
function h_gate(sites, i::Int)
    N = length(sites)
    h_mpo = MPO(sites)
    for j in 1:N
        h_mpo[j] = op("Id", sites[j])
    end
    h_mpo[i] = op("H", sites[i])
    return h_mpo
end

# RXX 

function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rxx_mpo = MPO(sites)
    
    # Assign identity operators to all sites
    for k in 1:N
        rxx_mpo[k] = op("Id", sites[k])
    end
    
    # Assign RXX operator with the specified angle ϕ to sites i and j
    rxx_mpo[i] = op("Rxx", sites[i], sites[j]; ϕ=ϕ)
    # rxx_mpo[j] = op("Rxx", sites[i], sites[j]; ϕ=ϕ) #I think we don't need this.
    
    return rxx_mpo
end

# CX Gate
function cx_gate(sites, control::Int, target::Int)
    N = length(sites)
    cx_mpo = MPO(sites)
    for j in 1:N
        cx_mpo[j] = op("Id", sites[j])
    end
    cx_mpo[control] = op("CX", sites[control], sites[target])

    return cx_mpo
end

# X Gate
function x_gate(sites, i::Int)
    N = length(sites)
    x_mpo = MPO(sites)
    for j in 1:N
        x_mpo[j] = op("Id", sites[j])
    end
    x_mpo[i] = op("X", sites[i])
    return x_mpo
end

# Rx Gate with angle θ
function rx_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rx_mpo = MPO(sites)
    for j in 1:N
        rx_mpo[j] = op("Id", sites[j])
    end
    rx_mpo[i] = op("Rx", sites[i]; θ=θ)
    return rx_mpo
end

# MPO for the 0 projector (|0⟩⟨0|)
function proj_0_mpo(sites, i::Int)
    N = length(sites)
    proj_mpo = MPO(sites)
    for j in 1:N
        proj_mpo[j] = op("Id", sites[j])
    end
    proj_mpo[i] = op("Proj0", sites[i])  # Projector onto |0⟩
    return proj_mpo
end

# MPO for the 1 projector (|1⟩⟨1|)
function proj_1_mpo(sites, i::Int)
    N = length(sites)
    proj_mpo = MPO(sites)
    for j in 1:N
        proj_mpo[j] = op("Id", sites[j])
    end
    proj_mpo[i] = op("Proj1", sites[i])  # Projector onto |1⟩
    return proj_mpo
end



proj_1_mpo (generic function with 1 method)

In [ ]:
# Rz Gate with angle θ
function rz_gate(sites, i::Int, θ::Float64)
    return op("Rz", sites[i]; θ=θ)
end

# Hadamard Gate
function h_gate(sites, i::Int)
    return op("H", sites[i])
end

# RXX Gate with angle ϕ
function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    return op("Rxx", sites[i], sites[j]; ϕ=ϕ)
end

# CX (Controlled-X) Gate
function cx_gate(sites, control::Int, target::Int)
    return op("CX", sites[control], sites[target])
end

# X Gate
function x_gate(sites, i::Int)
    return op("X", sites[i])
end

# Rx Gate with angle θ
function rx_gate(sites, i::Int, θ::Float64)
    return op("Rx", sites[i]; θ=θ)
end

# Projector onto |0⟩ (|0⟩⟨0|)
function proj_0_gate(sites, i::Int)
    return op("Proj0", sites[i])
end

# Projector onto |1⟩ (|1⟩⟨1|)
function proj_1_gate(sites, i::Int)
    return op("Proj1", sites[i])
end


proj_1_gate (generic function with 1 method)

Another way of defining MPOs

In [ ]:
using ITensors

# Rz Gate with angle θ
function rz_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rz_op = op("Rz", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rz_op = rz_op * op("Id", sites[k])
        end
    end
    return MPO(rz_op, sites)
end

# Hadamard Gate
function h_gate(sites, i::Int)
    N = length(sites)
    h_op = op("H", sites[i])
    for k in 1:N
        if k != i
            h_op = h_op * op("Id", sites[k])
        end
    end
    return MPO(h_op, sites)
end

# RXX Gate with angle ϕ
function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rxx_op = op("Rxx", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rxx_op = rxx_op * op("Id", sites[k])
        end
    end
    return MPO(rxx_op, sites)
end

# CX (Controlled-X) Gate
function cx_gate(sites, control::Int, target::Int)
    N = length(sites)
    cx_op = op("CX", sites[control], sites[target])
    for k in 1:N
        if k != control && k != target
            cx_op = cx_op * op("Id", sites[k])
        end
    end
    return MPO(cx_op, sites)
end

# X Gate
function x_gate(sites, i::Int)
    N = length(sites)
    x_op = op("X", sites[i])
    for k in 1:N
        if k != i
            x_op = x_op * op("Id", sites[k])
        end
    end
    return MPO(x_op, sites)
end

# Rx Gate with angle θ
function rx_gate(sites, i::Int, θ::Float64)
    N = length(sites)
    rx_op = op("Rx", sites[i]; θ=θ)
    for k in 1:N
        if k != i
            rx_op = rx_op * op("Id", sites[k])
        end
    end
    return MPO(rx_op, sites)
end

# Projector onto |0⟩ (|0⟩⟨0|)
function proj_0_gate(sites, i::Int)
    N = length(sites)
    proj0_op = op("Proj0", sites[i])
    for k in 1:N
        if k != i
            proj0_op = proj0_op * op("Id", sites[k])
        end
    end
    return MPO(proj0_op, sites)
end

# Projector onto |1⟩ (|1⟩⟨1|)
function proj_1_gate(sites, i::Int)
    N = length(sites)
    proj1_op = op("Proj1", sites[i])
    for k in 1:N
        if k != i
            proj1_op = proj1_op * op("Id", sites[k])
        end
    end
    return MPO(proj1_op, sites)
end

# function z_measurement(sites, i::Int)
#     N = length(sites)
#     z_meas = proj_0_gate(sites, i) - proj_1_gate(sites,i)
#     return z_meas
# end


proj_1_gate (generic function with 1 method)

In [ ]:

# check if the MPO creation was sucessful
function inspect_mpo(mpo::MPO, sites::Vector)
    N = length(sites) 

    println("\nManual Inspection of MPO:")
    for site_idx in 1:N
        println("Site $site_idx Operator:")
        println(mpo[site_idx])
    end
end

inspect_mpo (generic function with 1 method)

In [ ]:
# # Example usage
# N = 2
# sites1 = create_sites(N)
# θ = π / 4  
# rxx_MPO = cx_gate(sites1,1, 2)  
# inspect_mpo(rxx_MPO, sites1)

In [ ]:
function build_mpo_sequence(sites, gates)
    N = length(sites)
    mpo_list = []
    
    for gate in gates
        gate_type = gate[1]
        
        if gate_type == :rxx
            ϕ = gate[2]
            i, j = gate[3], gate[4]
            push!(mpo_list, rxx_gate(sites, i, j, ϕ))
        
        elseif gate_type == :rz
            θ = gate[2]
            i = gate[3]
            push!(mpo_list, rz_gate(sites, i, θ))
        
        elseif gate_type == :h
            i = gate[2]
            push!(mpo_list, h_gate(sites, i))
        
        elseif gate_type == :x
            i = gate[2]
            push!(mpo_list, x_gate(sites, i))
        
        elseif gate_type == :rx
            θ = gate[2]
            i = gate[3]
            push!(mpo_list, rx_gate(sites, i, θ))
        
        elseif gate_type == :cx
            control = gate[2]
            target = gate[3]
            push!(mpo_list, cx_gate(sites, control, target))
        
        elseif gate_type == :measure_qpd
            i = gate[2]
            push!(mpo_list, proj_0_gate(sites, i))
            push!(mpo_list, -proj_1_gate(sites,i))
        end
    end
    
    return mpo_list
end


build_mpo_sequence (generic function with 1 method)

In [ ]:
# Ensure 'sites' is used consistently
sites = create_sites(n_qubits)

# Build the MPO sequence with the same 'sites'
sequence = build_mpo_sequence(sites, gates);
# Create the MPS using the same 'sites'
psi = productMPS(sites, "0")
rho = outer(psi,psi')
maxdim1 = 1000;  # Adjust depending on memory constraints
cutoff1 = 1e-12;

In [ ]:

for mpo in sequence
    println("Applying MPO...")
    
    # Ensure the MPO and MPS share the same site indices
    rho = apply(apply(mpo, rho), dag(mpo))
end


LoadError: DimensionMismatch: lengths of MPOs A (5) and B (1) do not match

In [ ]:
function rxx_gate(sites, i::Int, j::Int, ϕ::Float64)
    N = length(sites)
    rxx_op = op("Rxx", sites[i], sites[j]; ϕ=ϕ)
    for k in 1:N
        if k != i && k != j
            rxx_op = rxx_op * op("Id", sites[k])
        end
    end
    return MPO(rxx_op, sites)
end

rxx_gate (generic function with 1 method)

In [ ]:
gates

13-element Vector{Any}:
 (:rxx, 0.4, 1, 2)
 (:rxx, 0.4, 2, 3)
 (:rz, 0.4, 2)
 (:rxx, 0.4, 3, 4)
 (:rz, 0.4, 3)
 (:rxx, 0.4, 4, 5)
 (:rxx, 0.4, 1, 5)
 (:h, 5)
 (:measure_qpd, 5)
 (:h, 5)
 (:rz, 0.4, 1)
 (:rz, 0.4, 4)
 (:rz, 0.4, 5)

In [ ]:
inspect_mpo(mpos_test[], sites)


Manual Inspection of MPO:
Site 1 Operator:
ITensor ord=2
Dim 1: (dim=2|id=622|"Qubit,i1")'
Dim 2: (dim=2|id=622|"Qubit,i1")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.7071067811865475   0.7071067811865475
 0.7071067811865475  -0.7071067811865475


In [ ]:
n_qubits = 3
sites = create_sites(n_qubits)
gates_test = [(:h, i) for i in 1:n_qubits]
push!(gates_test, (:measure_qpd, 1))
mpos_test = build_mpo_sequence(sites, gates_test);

psi = productMPS(sites, "0")
rho = outer(psi,psi')


Z_sum_op = ITensor()

os = OpSum()
for i in 1:n_qubits
    os += "X", i
end
Zmpo = MPO(os, sites)

rho1 = apply(apply(mpos_test[1], rho), mpos_test[1])
for mpo in mpos_test[2:end]
    # Ensure the MPO and MPS share the same site indices
    rho1 = rho1 + apply(apply(mpo, rho), dag(mpo))
end

In [ ]:
inner(rho, Zmpo)

0.0